In [3]:
#!/usr/bin/env python
# coding: utf-8

# ===================================================================
# 車禍預測比賽 - PyTorch 多模態模型預測腳本
# 說明: 此腳本用於載入已訓練的模型，並對測試集進行預測。
# ===================================================================

# --- 0. 匯入必要套件 ---
import os
import pandas as pd
import numpy as np
import re
import joblib
from datetime import datetime
from glob import glob
from tqdm import tqdm
import random
import gc

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm

# ===================================================================
#                        *** 1. 設定區 ***
# ===================================================================
# !!! 重要 !!!
# 此處的 Config 必須與您用來訓練的 VLM_5X3_2階段(065).py 腳本中的設定完全一致
# 才能正確載入模型架構與處理資料。
class Config:
    # --- 1. 路徑、輸出與環境設定 ---
    BASE_IMG_PATH = '.'                   # 圖像資料夾 (road, freeway) 的根路徑
    RANDOM_STATE = 42                     # 固定隨機種子以確保實驗可重複
    
    # --- 2. 動態採樣策略設定 ---
    WINDOW_SIZE = 5                       # 每個採樣窗口要包含多少張圖片
    NUM_WINDOWS = 3                       # 總共要建立多少個採樣窗口 (組)
    
    # --- 3. 資料處理超參數 ---
    MAX_IMG_SEQ_LEN = WINDOW_SIZE         # 圖像序列長度現在由窗口大小決定
    MAX_TEXT_LEN = 512                    # VLM 文字序列的最大長度
    IMG_SIZE = (224, 224)                 # 圖像模型的標準輸入尺寸
    VOCAB_SIZE = 8000                     # 文字字典大小
    
    # --- 4. 模型架構超參數 ---
    MODEL_NAME = 'vit_base_patch16_clip_224.openai' # 使用的預訓練圖像模型
    EMBEDDING_DIM = 128                   # 文字嵌入層的維度
    TEXT_LSTM_UNITS = 64                  # 處理文字的 LSTM 神經元數量
    IMAGE_LSTM_UNITS = 64                 # 處理圖像序列的 LSTM 神經元數量
    DENSE_UNITS = 128                     # 融合後的全連接層神經元數量
    DROPOUT_RATE = 0.5                    # Dropout 層的比率
    
    # --- 5. 訓練過程超參數 (預測時用不到，但保留以示完整) ---
    NUM_SPLITS = 5                        # 交叉驗證的摺數
    BATCH_SIZE = 8                        # 批次大小 (預測時可以根據 VRAM 調整)

# ===================================================================
#                  *** 2. 輔助工具與類別定義 ***
# ===================================================================
# !!! 重要 !!!
# 這些類別與函式也必須與原訓練腳本完全相同。

def set_environment(seed):
    torch.manual_seed(seed);
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False
    np.random.seed(seed); random.seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n[環境資訊] 隨機種子已設定為: {seed}"); print(f"[環境資訊] 執行設備: {device}")
    if torch.cuda.is_available(): print(f"[環境資訊] GPU 名稱: {torch.cuda.get_device_name(0)}")
    return device

class SimpleTokenizer:
    def __init__(self, num_words=8000, oov_token="<OOV>"): self.word_index, self.num_words, self.oov_token = {oov_token: 1}, num_words, oov_token
    def fit_on_texts(self, texts):
        word_counts = pd.Series([word for text in texts for word in text.split()]).value_counts()
        for i, (word, _) in enumerate(word_counts.head(self.num_words - 2).items()): self.word_index[word] = i + 2
    def texts_to_sequences(self, texts): return [[self.word_index.get(word, 1) for word in text.split()] for text in texts]

class DashcamDataset(Dataset):
    def __init__(self, df, tokenizer, config, transform, sampling_strategy, is_test=False):
        self.df, self.tokenizer, self.config, self.transform, self.sampling_strategy, self.is_test = df, tokenizer, config, transform, sampling_strategy, is_test
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]; text_seq = self.tokenizer.texts_to_sequences([row['processed_text']])[0]
        padded_text = np.zeros(self.config.MAX_TEXT_LEN, dtype=np.int64)
        if len(text_seq) > 0:
            if len(text_seq) > self.config.MAX_TEXT_LEN: text_seq = text_seq[-self.config.MAX_TEXT_LEN:]
            padded_text[-len(text_seq):] = text_seq
        image_sequence = torch.zeros(self.config.MAX_IMG_SEQ_LEN, 3, *self.config.IMG_SIZE)
        all_paths = row['image_paths']
        start_idx, end_idx = self.sampling_strategy['start'], self.sampling_strategy['end']
        paths_to_load = all_paths[start_idx:end_idx] if len(all_paths) >= abs(start_idx) else []
        for j, img_path in enumerate(paths_to_load):
            if j >= self.config.MAX_IMG_SEQ_LEN: break
            try: img = Image.open(img_path).convert('RGB'); image_sequence[j] = self.transform(img)
            except Exception: continue
        # 預測時只返回檔名，不返回標籤
        return torch.tensor(padded_text, dtype=torch.long), image_sequence, row['file_name']

class MultimodalModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config; self.embedding = nn.Embedding(config.VOCAB_SIZE, config.EMBEDDING_DIM, padding_idx=0)
        self.text_lstm = nn.LSTM(config.EMBEDDING_DIM, config.TEXT_LSTM_UNITS, batch_first=True, bidirectional=True); self.text_dropout = nn.Dropout(config.DROPOUT_RATE)
        self.image_encoder = timm.create_model(config.MODEL_NAME, pretrained=False, num_classes=0, global_pool='avg') # pretrained=False, 因為我們要載入自己的權重
        self.image_lstm = nn.LSTM(self.image_encoder.num_features, config.IMAGE_LSTM_UNITS, batch_first=True, bidirectional=True); self.image_dropout = nn.Dropout(config.DROPOUT_RATE)
        fusion_dim = (config.TEXT_LSTM_UNITS * 2) + (config.IMAGE_LSTM_UNITS * 2)
        self.classifier = nn.Sequential(nn.Linear(fusion_dim, config.DENSE_UNITS), nn.ReLU(), nn.Dropout(config.DROPOUT_RATE), nn.Linear(config.DENSE_UNITS, 2))
        print(f"PyTorch 多模態模型 ({config.MODEL_NAME}) 架構已建立。")
    def forward(self, text, image):
        text_embeds = self.embedding(text); _, (text_hidden, _) = self.text_lstm(text_embeds); text_features = torch.cat((text_hidden[-2,:,:], text_hidden[-1,:,:]), dim=1); text_features = self.text_dropout(text_features)
        b, t, c, h, w = image.shape; img_flat = image.view(b * t, c, h, w); img_features = self.image_encoder(img_flat); img_features = img_features.view(b, t, -1); _, (img_hidden, _) = self.image_lstm(img_features); img_features = torch.cat((img_hidden[-2,:,:], img_hidden[-1,:,:]), dim=1); img_features = self.image_dropout(img_features)
        return self.classifier(torch.cat((text_features, img_features), dim=1))

# ===================================================================
#                     *** 3. 主預測流程 ***
# ===================================================================
def predict():
    # --- 設定 ---
    config = Config()
    device = set_environment(config.RANDOM_STATE)
    
    # 指定已訓練模型的根目錄
    TRAINED_MODEL_DIR = 'output_pytorch_MultiWindow_20250710_131200'
    if not os.path.isdir(TRAINED_MODEL_DIR):
        print(f"[錯誤] 找不到訓練好的模型資料夾: {TRAINED_MODEL_DIR}")
        print("請確認此腳本與該資料夾位於同一目錄下。")
        return
        
    # 建立一個新的資料夾來存放本次預測的結果
    TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
    INFERENCE_OUTPUT_DIR = f'inference_results_{TIMESTAMP}'
    os.makedirs(INFERENCE_OUTPUT_DIR, exist_ok=True)
    print(f"本次預測結果將儲存於: {INFERENCE_OUTPUT_DIR}")

    # --- 重建採樣策略 ---
    config.SAMPLING_STRATEGIES = []
    for i in range(1, config.NUM_WINDOWS + 1):
        start_offset = (i - 1) * config.WINDOW_SIZE + 1
        end_offset = i * config.WINDOW_SIZE
        config.SAMPLING_STRATEGIES.append({
            'name': f'Last_{start_offset}_to_{end_offset}',
            'start': -end_offset,
            'end': None if i == 1 else -(end_offset - config.WINDOW_SIZE)
        })
    print("\n" + "="*15, "已重建的採樣策略", "="*15); [print(s) for s in config.SAMPLING_STRATEGIES]; print("-" * 50)
    
    # --- 步驟 A: 準備測試資料框架 (與訓練時相同) ---
    print("\n" + "="*15, "步驟 A: 準備測試資料", "="*15)
    def get_paths_and_counts(df, base_path_map):
        img_paths_list = [sorted(glob(os.path.join(base_path_map['road' if row['file_name'].startswith('road') else 'freeway'], row['file_name'], '*.jpg'))) or sorted(glob(os.path.join(base_path_map['road' if row['file_name'].startswith('road') else 'freeway'], row['file_name'], '*.JPG'))) for _, row in df.iterrows()]
        df['image_paths'] = img_paths_list; df['image_count'] = [len(p) for p in img_paths_list]
        return df

    test_captions_df = pd.concat([pd.read_csv('road_test_with_captions.csv'), pd.read_csv('freeway_test_with_captions.csv')], ignore_index=True)
    sub_df = pd.read_csv('sample_submission.csv')
    test_df = pd.merge(sub_df[['file_name']], test_captions_df, on='file_name', how='left')
    test_base_path_map = {'road': os.path.join(config.BASE_IMG_PATH, 'road', 'test'), 'freeway': os.path.join(config.BASE_IMG_PATH, 'freeway', 'test')}
    test_df = get_paths_and_counts(test_df, test_base_path_map)
    def preprocess_text(text): return ' '.join([re.sub(r'[^a-z0-9\s]', '', s.lower()).strip() for s in str(text).split('|')])
    test_df['processed_text'] = test_df['captions'].apply(preprocess_text)
    
    # --- 載入 Tokenizer ---
    tokenizer_path = os.path.join(TRAINED_MODEL_DIR, 'tokenizer.pkl')
    try:
        tokenizer = joblib.load(tokenizer_path)
        print("Tokenizer 已從 " + tokenizer_path + " 載入。")
    except FileNotFoundError:
        print(f"[錯誤] 找不到 Tokenizer 檔案: {tokenizer_path}")
        return
    print("-" * 50)
    
    # --- 定義圖像轉換 ---
    transform = transforms.Compose([
        transforms.Resize(config.IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # --- 步驟 B: 進行集成預測 ---
    grand_ensemble_test_probs = []
    
    for strategy in config.SAMPLING_STRATEGIES:
        strategy_name = strategy['name']
        print(f"\n" + "#"*25, f"開始處理策略: {strategy_name}", "#"*25)
        
        STRATEGY_MODEL_DIR = os.path.join(TRAINED_MODEL_DIR, strategy_name)
        
        test_dataset = DashcamDataset(test_df, tokenizer, config, transform, sampling_strategy=strategy, is_test=True)
        test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)
        
        strategy_fold_probs = []
        for fold in range(config.NUM_SPLITS):
            # 在訓練腳本中，兩階段訓練最終儲存的模型名稱是 temp_best_head_fold_{fold+1}.pth
            # 如果您使用的是單階段，請將 model_filename 改為 'best_model_fold_{fold+1}.pth'
            model_filename = f'temp_best_head_fold_{fold+1}.pth'
            model_path = os.path.join(STRATEGY_MODEL_DIR, model_filename)

            if not os.path.exists(model_path):
                print(f"[警告] 在策略 [{strategy_name}] 中找不到模型檔案: {model_path}，跳過此 Fold。")
                continue
            
            # 載入模型
            model = MultimodalModel(config).to(device)
            model.load_state_dict(torch.load(model_path, map_location=device))
            model.eval()
            
            fold_probs = []
            with torch.no_grad():
                for text, image, _ in tqdm(test_loader, desc=f"預測中 (策略: {strategy_name}, Fold: {fold+1})"):
                    outputs = model(text.to(device), image.to(device))
                    # 使用 softmax 取得機率，並選取 "risk=1" 的機率
                    probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                    fold_probs.extend(probs)
            
            strategy_fold_probs.append(fold_probs)
            del model; gc.collect(); torch.cuda.empty_cache()

        if strategy_fold_probs:
            # 對此策略內的所有 Fold 結果取平均
            strategy_final_probs = np.mean(strategy_fold_probs, axis=0)
            grand_ensemble_test_probs.append(strategy_final_probs)
            print(f"✅ 策略 [{strategy_name}] 預測完成。")
        else:
            print(f"❌ 策略 [{strategy_name}] 沒有找到任何模型，無法進行預測。")

    # --- 步驟 C: 最終集成與儲存結果 ---
    if not grand_ensemble_test_probs:
        print("\n" + "="*25, "預測失敗", "="*25)
        print("❌ 未能從任何策略中成功載入模型並產生預測。請檢查 TRAINED_MODEL_DIR 路徑與模型檔案是否存在。")
    else:
        print("\n" + "="*25, "開始終極集成預測", "="*25)
        
        # 對所有策略的平均結果再次取平均
        grand_final_probs = np.mean(grand_ensemble_test_probs, axis=0)
        grand_final_predictions = (grand_final_probs > 0.5).astype(int)
        
        # 準備最終的 DataFrame
        final_submission_df = pd.DataFrame({
            'file_name': test_df['file_name'].tolist(),
            'probability': grand_final_probs, # 新增的機率欄位
            'risk': grand_final_predictions
        })
        
        # 1. 儲存重現的 submission 檔 (只有 file_name, risk)
        repro_submission_path = os.path.join(INFERENCE_OUTPUT_DIR, 'submission_reproduced.csv')
        final_submission_df[['file_name', 'risk']].to_csv(repro_submission_path, index=False)
        print(f"\n✅ 重現的預測結果已儲存至: {repro_submission_path}")
        print("預測分佈:")
        print(final_submission_df['risk'].value_counts())
        
        # 2. 儲存包含機率的 submission 檔
        prob_submission_path = os.path.join(INFERENCE_OUTPUT_DIR, 'submission_probabilities.csv')
        final_submission_df.to_csv(prob_submission_path, index=False)
        print(f"\n✅✅✅ 包含機率的預測結果已儲存至: {prob_submission_path}")
        
    print("\n🎉🎉🎉 所有預測流程執行完畢！🎉🎉🎉")

if __name__ == '__main__':
    predict()


[環境資訊] 隨機種子已設定為: 42
[環境資訊] 執行設備: cuda
[環境資訊] GPU 名稱: NVIDIA GeForce RTX 3080
本次預測結果將儲存於: inference_results_20250719_001815

=============== 已重建的採樣策略 ===============
{'name': 'Last_1_to_5', 'start': -5, 'end': None}
{'name': 'Last_6_to_10', 'start': -10, 'end': -5}
{'name': 'Last_11_to_15', 'start': -15, 'end': -10}
--------------------------------------------------

=============== 步驟 A: 準備測試資料 ===============
Tokenizer 已從 output_pytorch_MultiWindow_20250710_131200\tokenizer.pkl 載入。
--------------------------------------------------

######################### 開始處理策略: Last_1_to_5 #########################
PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 架構已建立。


C:\Users\wolves9\AppData\Local\Temp\ipykernel_12856\1005008649.py:210: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locatio

PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 架構已建立。


預測中 (策略: Last_1_to_5, Fold: 2): 100%|█████████████████████████████████████████████| 31/31 [00:17<00:00,  1.79it/s]


PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 架構已建立。


...

PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 架構已建立。


預測中 (策略: Last_11_to_15, Fold: 4): 100%|███████████████████████████████████████████| 31/31 [00:17<00:00,  1.81it/s]


PyTorch 多模態模型 (vit_base_patch16_clip_224.openai) 架構已建立。


預測中 (策略: Last_11_to_15, Fold: 5): 100%|███████████████████████████████████████████| 31/31 [00:17<00:00,  1.81it/s]

✅ 策略 [Last_11_to_15] 預測完成。

========================= 開始終極集成預測 =========================

✅ 重現的預測結果已儲存至: inference_results_20250719_001815\submission_reproduced.csv
預測分佈:
risk
0    149
1     94
Name: count, dtype: int64

✅✅✅ 包含機率的預測結果已儲存至: inference_results_20250719_001815\submission_probabilities.csv

🎉🎉🎉 所有預測流程執行完畢！🎉🎉🎉
